# LIBRARY IMPORTS

In [26]:
%run importing.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# LOAD DATA

In [27]:
df1 = pd.read_csv('train.csv')
df1.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


# DATA PREPARATION PIPELINE

In [3]:
df2, quants, quals = fill_missing(df1)
df2.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,MISSING,Reg,Lvl,AllPub,...,0,MISSING,MISSING,MISSING,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,MISSING,Reg,Lvl,AllPub,...,0,MISSING,MISSING,MISSING,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,MISSING,IR1,Lvl,AllPub,...,0,MISSING,MISSING,MISSING,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,MISSING,IR1,Lvl,AllPub,...,0,MISSING,MISSING,MISSING,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,MISSING,IR1,Lvl,AllPub,...,0,MISSING,MISSING,MISSING,0,12,2008,WD,Normal,250000


In [4]:
df2.isna().sum().sum()

0

In [5]:
df3, ohe = one_hot(df2, quals)
df3.head()

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_MISSING,Alley_Pave,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [6]:
df4 = pd.concat([df2, df3], axis=1)
df4.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,RL,65.0,8450,Pave,MISSING,Reg,Lvl,AllPub,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2,20,RL,80.0,9600,Pave,MISSING,Reg,Lvl,AllPub,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,3,60,RL,68.0,11250,Pave,MISSING,IR1,Lvl,AllPub,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,4,70,RL,60.0,9550,Pave,MISSING,IR1,Lvl,AllPub,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,5,60,RL,84.0,14260,Pave,MISSING,IR1,Lvl,AllPub,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [7]:
y1 = df4.SalePrice
y2 = np.log(y1)
y2.head()

0    12.247694
1    12.109011
2    12.317167
3    11.849398
4    12.429216
Name: SalePrice, dtype: float64

In [8]:
for c in ['Id', 'SalePrice']:
    quants.remove(c)

In [9]:
X1 = df4[quants + df3.columns.tolist()]
X1.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
scaler = StandardScaler().fit(X1)
X2 = scaler.transform(X1)
X2.shape

(1460, 304)

# SPLIT DATA

In [11]:
X = X2
y = y2
split_size = 0.25
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=split_size, random_state=42)
X_train.shape, X_val.shape

((1095, 304), (365, 304))

# MODELING

In [12]:
lin_params = {}
lin_grid = GridSearchCV(LinearRegression(), lin_params, cv=5, n_jobs=-1).fit(X_train, y_train)
lin = LinearRegression(**lin_grid.best_params_).fit(X_train, y_train)

In [13]:
lso_params = {'alpha': np.logspace(-4, 0, 5)}
lso_grid = GridSearchCV(Lasso(), lso_params, cv=5, n_jobs=-1).fit(X_train, y_train)
lso = Lasso(**lso_grid.best_params_).fit(X_train, y_train)

In [14]:
rdg_params = {'alpha': np.logspace(-4, 0, 5)}
rdg_grid = GridSearchCV(Ridge(), rdg_params, cv=5, n_jobs=-1).fit(X_train, y_train)
rdg = Ridge(**rdg_grid.best_params_).fit(X_train, y_train)

In [15]:
knn_params = {'n_neighbors': range(1,10), 
              'weights': ['uniform', 'distance'], 
              'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute']}
knn_grid = GridSearchCV(KNeighborsRegressor(), knn_params, cv=5, n_jobs=-1).fit(X_train, y_train)
knn = KNeighborsRegressor(**knn_grid.best_params_).fit(X_train, y_train)

In [18]:
etr_params = {'n_estimators': range(100, 501, 100), 
              'max_depth': range(1,10)}
etr_grid = GridSearchCV(ExtraTreesRegressor(), etr_params, cv=5, n_jobs=-1).fit(X_train, y_train)
etr = ExtraTreesRegressor(**etr_grid.best_params_).fit(X_train, y_train)

In [21]:
rfr_params = {'n_estimators': range(100, 501, 100), 
              'max_depth': range(1,10)}
rfr_grid = GridSearchCV(RandomForestRegressor(), rfr_params, cv=5, n_jobs=-1).fit(X_train, y_train)
rfr = RandomForestRegressor(**rfr_grid.best_params_).fit(X_train, y_train)

In [22]:
abr_params = {'n_estimators': range(100, 501, 100), 
              'learning_rate': np.logspace(-4, 0, 5)}
abr_grid = GridSearchCV(AdaBoostRegressor(), abr_params, cv=5, n_jobs=-1).fit(X_train, y_train)
abr = AdaBoostRegressor(**abr_grid.best_params_).fit(X_train, y_train)

In [23]:
gbr_params = {'n_estimators': range(100, 501, 100), 
              'learning_rate': np.logspace(-4, 0, 5),
              'max_depth': range(1, 10)}
gbr_grid = GridSearchCV(GradientBoostingRegressor(), gbr_params, cv=5, n_jobs=-1).fit(X_train, y_train)
gbr = GradientBoostingRegressor(**gbr_grid.best_params_).fit(X_train, y_train)

In [24]:
cbr = CatBoostRegressor(verbose=False, iterations=2000).fit(X_train, y_train)

In [29]:
stk = StackingCVRegressor(regressors=(lso, knn, etr, rfr, abr, gbr, cbr),
                                meta_regressor=rdg,
                                use_features_in_secondary=True)
stk.fit(X_train, y_train)

StackingCVRegressor(cv=5,
          meta_regressor=Ridge(alpha=0.0001, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
          n_jobs=None, pre_dispatch='2*n_jobs', random_state=None,
          refit=True,
          regressors=(Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False), KNeighborsRegressor(algorithm='brute', leaf_size=30, metric='minkowski',
          metric_param...ction=0.1, verbose=0, warm_start=False), <catboost.core.CatBoostRegressor object at 0x7f0f77d455f8>),
          shuffle=True, store_train_meta_features=False,
          use_features_in_secondary=True, verbose=0)

In [33]:
print('lin:', mean_squared_error(y_val, lin.predict(X_val)) ** 0.5)
print('lso:', mean_squared_error(y_val, lso.predict(X_val)) ** 0.5)
print('rdg:', mean_squared_error(y_val, rdg.predict(X_val)) ** 0.5)
print('knn:', mean_squared_error(y_val, knn.predict(X_val)) ** 0.5)
print('etr:', mean_squared_error(y_val, etr.predict(X_val)) ** 0.5)
print('rfr:', mean_squared_error(y_val, rfr.predict(X_val)) ** 0.5)
print('abr:', mean_squared_error(y_val, abr.predict(X_val)) ** 0.5)
print('gbr:', mean_squared_error(y_val, gbr.predict(X_val)) ** 0.5)
print('cbr:', mean_squared_error(y_val, cbr.predict(X_val)) ** 0.5)
print('stk:', mean_squared_error(y_val, stk.predict(X_val)) ** 0.5)

lin: 2284748619831.9175
lso: 0.12803818933618355
rdg: 0.12720983458407695
knn: 0.20353855651338118
etr: 0.15027988948903057
rfr: 0.14130645298717562
abr: 0.17840109297262965
gbr: 0.13622797803011777
cbr: 0.2096480431701631
stk: 0.131371114526011


In [ ]:
# model = Sequential()
# model.add(Dense(units=2096, activation='relu', input_shape=(265,)))
# model.add(Dropout(0.25))
# model.add(Dense(units=1024, activation='relu'))
# model.add(Dropout(0.25))
# model.add(Dense(units=1, activation='linear'))
# opt = optimizers.SGD()
# model.compile(loss='mean_squared_error', optimizer=opt, metrics=[rmse])
# model.summary()

In [ ]:
# history = model.fit(X_train, y_train, epochs=201, batch_size=25, verbose=0, validation_split=0.3,
#                     shuffle=True, callbacks=[Logger(50)])

In [ ]:
# model = GradientBoostingRegressor(n_estimators=400, max_depth=3).fit(pX, py)

In [ ]:
# df_kaggle = pd.read_csv('test.csv')
# X_kaggle, _ = eda(df_kaggle, False, more_more_more)
# X_kaggle, _ = process(X_kaggle, enter_scalar)

In [ ]:
# dict_kaggle = {'Id' : df_kaggle['Id'], 'SalePrice': np.exp(model.predict(X_kaggle))}
# pd.DataFrame(dict_kaggle).to_csv('predictions.csv', index=False)